In [5]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")

url1 = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"


url_stopId = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={API_KEY}"

url_stopName = f"https://api.resrobot.se/v2.1/location.name?input=Göteborg&format=json&accessId={API_KEY}"

url_arrivals = f"https://api.resrobot.se/v2.1/arrivalBoard?id=740000002&format=json&accessId={API_KEY}"

response = requests.get(url_stopName)
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [12]:
result["stopLocationOrCoordLocation"][0]["StopLocation"]["name"]

'GÖTEBORG'

In [14]:
def get_location_name(location):
    url_stopName = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    result_name = requests.get(url_stopName)
    return result_name.json().get("stopLocationOrCoordLocation")

location_name = get_location_name("Trätorget")
print(location_name)

[{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '128'}], 'timezoneOffset': 60, 'id': 'A=1@O=Trätorget (Göteborg kn)@X=12053177@Y=57715439@U=1@L=740059660@B=1@p=1737347805@', 'extId': '740059660', 'name': 'Trätorget (Göteborg kn)', 'lon': 12.053177, 'lat': 57.715439, 'weight': 533, 'products': 128, 'minimumChangeDuration': 'PT0S'}}, {'CoordLocation': {'links': [{'link': [{'rel': 'refine', 'href': 'https://api.resrobot.se/v2.1/location.name?input=Tr%C3%A4torget%2C+Falk%C3%B6ping&refineId=A%3D2%40O%3DTr%C3%A4torget%2C+Falk%C3%B6ping%40X%3D13554412%40Y%3D58160352%40U%3D174%40b%3D980141995%40B%3D1%40p%3D1479298166%40&type=A'}]}], 'icon': {'res': 'loc_addr'}, 'id': 'A=2@O=Trätorget, Falköping@X=13554412@Y=58160352@U=174@b=980141995@B=1@p=1479298166@', 'name': 'Trätorget, Falköping', 'type': 'ADR', 'lon': 13.554412, 'lat': 58.160352, 'refinable': True}}, {'CoordLocation': {'links': [{'link': [{'rel': 'refine', 'href': 'https://api.resrobot.se/v2.1/location.name?input

In [15]:
location_name[0].get("StopLocation")

{'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '128'}],
 'timezoneOffset': 60,
 'id': 'A=1@O=Trätorget (Göteborg kn)@X=12053177@Y=57715439@U=1@L=740059660@B=1@p=1737347805@',
 'extId': '740059660',
 'name': 'Trätorget (Göteborg kn)',
 'lon': 12.053177,
 'lat': 57.715439,
 'weight': 533,
 'products': 128,
 'minimumChangeDuration': 'PT0S'}

In [16]:
station_id = location_name[0].get("StopLocation", {}).get("extId")
print(station_id)

740059660


In [17]:
def get_departures(location_ids):
    url_departures = f"https://api.resrobot.se/v2.1/departureBoard?id={location_ids}&format=json&accessId={API_KEY}"
    response = requests.get(url_departures)
    data = response.json()
    departures = data.get("Departure", [])
    return departures

In [18]:
import re
station = get_location_name("Göteborg C")
def get_timetable():
    if station:
        station_name = station[0]["StopLocation"]["name"]
        station_id_raw = station[0]["StopLocation"]["id"]

        match = re.search(r"L=(\d+)", station_id_raw)
        if match:
            station_id = match.group(1)
        else:
            print("Kunde inte extrahera stationens ID.")
            station_id = None
        print(f"Stationens namn: {station_name}")
        print(f"Stations id: {station_id}")

        departures = get_departures(station_id)
        if departures:
            for departure in departures:
                print(f"Tid: {departure['time']}, Destination: {departure['direction']}")

            num_dep = len(departures)
            print(f"\nAntal transporter som avgår från {station_name}: {num_dep}")
        else:
            print("Inga avgångar")
    else:
        print("Stationen hittades inte")

get_timetable()

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 09:15:00, Destination: Göteborg Eketrägatan
Tid: 09:15:00, Destination: Kungsbacka station
Tid: 09:15:00, Destination: Göteborg Varmfrontsgatan
Tid: 09:16:00, Destination: Kungälv resecentrum
Tid: 09:16:00, Destination: Göteborg Lindholmen
Tid: 09:16:00, Destination: Göteborg Östra sjukhuset
Tid: 09:16:00, Destination: Kortedala Aprilgatan (Göteborg kn)
Tid: 09:16:00, Destination: Göteborg Centralst Drottningt
Tid: 09:17:00, Destination: Mölnlycke station (Härryda kn)
Tid: 09:18:00, Destination: Kungälv resecentrum
Tid: 09:18:00, Destination: Stockholm Centralstation
Tid: 09:18:00, Destination: Göteborg Heden
Tid: 09:18:00, Destination: Amhult Resecentrum (Göteborg kn)
Tid: 09:19:00, Destination: Mölnlycke station (Härryda kn)
Tid: 09:19:00, Destination: Partille centrum
Tid: 09:20:00, Destination: Bockkranen (Göteborg kn)
Tid: 09:20:00, Destination: Partille centrum
Tid: 09:20:00, Destination: Älvängen station (Ale k

In [20]:
station = get_location_name(input("Hållplats: "))
get_timetable()

Stationens namn: Stabbetorget (Göteborg kn)
Stations id: 740059587
Tid: 12:13:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 12:14:00, Destination: Göteborg Östra sjukhuset
Tid: 12:23:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 12:24:00, Destination: Göteborg Östra sjukhuset
Tid: 12:33:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 12:34:00, Destination: Göteborg Östra sjukhuset
Tid: 12:43:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 12:44:00, Destination: Göteborg Östra sjukhuset
Tid: 12:53:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 12:54:00, Destination: Göteborg Östra sjukhuset
Tid: 13:03:00, Destination: Hinnebäcksgatan (Göteborg kn)
Tid: 13:04:00, Destination: Göteborg Östra sjukhuset

Antal transporter som avgår från Stabbetorget (Göteborg kn): 12


In [22]:
def get_arrivials(location_ids):
    url_arrivals = f"https://api.resrobot.se/v2.1/arrivalBoard?id={location_ids}&format=json&accessId={API_KEY}"
    response = requests.get(url_arrivals)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None
    
    data_arrivals = response.json()
    arrivals = data_arrivals.get("Arrival", [])
    return arrivals
get_arrivials("Trätorget")

Error: 400
{"serverVersion":"2.45.1","dialectVersion":"2.45","errorCode":"SVC_LOC","errorText":"location missing or invalid (LOCATION).","internalErrorCode":"LOCATION","internalErrorText":"HCI Service: location missing or invalid","internalErrorTextOut":"Det uppstod ett internt fel under sökningen","requestId":"default-request-id"}


In [23]:
station = get_location_name("Göteborg C")
def get_timetable_arrivals():
    if station:
        station_name = station[0]["StopLocation"]["name"]
        station_id_raw = station[0]["StopLocation"]["id"]

        match = re.search(r"L=(\d+)", station_id_raw)
        if match:
            station_id = match.group(1)
        else:
            print("Kunde inte extrahera stationens ID.")
            station_id = None
        print(f"Stationens namn: {station_name}")
        print(f"Stations id: {station_id}")

        arrivals = get_arrivials(station_id)
        if arrivals:
            print(f"Ankomster till {station_name}:")
            for arrival in arrivals:
                time = arrival.get('time', 'N/A')
                origin = arrival.get('origin', 'N/A')
                transport = arrival.get('ProductAtStop', {}).get('displayNumber', 'Okänt fordon')
                print(f"Tid: {time}, Avgång: {origin}, FordonsNr: {transport}")

            num_arrivals = len(arrivals)
            print(f"\nAntal transporter som anländer till {station_name}: {num_arrivals}")
        else:
            print("Inga avgångar")
    else:
        print("Stationen hittades inte")

get_timetable_arrivals()

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Ankomster till Göteborg Centralstation:
Tid: 12:12:00, Avgång: Göteborg Eketrägatan, FordonsNr: 21
Tid: 12:13:00, Avgång: Merkuriusgatan (Göteborg kn), FordonsNr: 21
Tid: 12:14:00, Avgång: Höga hallar (Härryda kn), FordonsNr: X4
Tid: 12:14:00, Avgång: Partille centrum, FordonsNr: X1
Tid: 12:14:00, Avgång: Göteborg Heden, FordonsNr: 513
Tid: 12:15:00, Avgång: Höga hallar (Härryda kn), FordonsNr: X4
Tid: 12:15:00, Avgång: Kungsbacka station, FordonsNr: 3046
Tid: 12:16:00, Avgång: Hinnebäcksgatan (Göteborg kn), FordonsNr: 17
Tid: 12:16:00, Avgång: Göteborg Varmfrontsgatan, FordonsNr: 6
Tid: 12:17:00, Avgång: Kungälv resecentrum, FordonsNr: X4
Tid: 12:17:00, Avgång: Partille centrum, FordonsNr: 510
Tid: 12:18:00, Avgång: Kullavik hamn (Kungsbacka kn), FordonsNr: X3
Tid: 12:18:00, Avgång: Sävedalen Ljungkullen (Partille kn), FordonsNr: SVART
Tid: 12:19:00, Avgång: Kungälv resecentrum, FordonsNr: X4
Tid: 12:19:00, Avgång: Bockkr

In [24]:
station = get_location_name("Göteborg C")

get_timetable()

Stationens namn: Göteborg Centralstation
Stations id: 740000002
Tid: 12:12:00, Destination: Merkuriusgatan (Göteborg kn)
Tid: 12:13:00, Destination: Göteborg Eketrägatan
Tid: 12:14:00, Destination: Kungälv resecentrum
Tid: 12:14:00, Destination: Hornkamsgatan (Göteborg kn)
Tid: 12:14:00, Destination: Partille centrum
Tid: 12:15:00, Destination: Kungälv resecentrum
Tid: 12:15:00, Destination: Oslo S (Norge)
Tid: 12:15:00, Destination: Oslo Bussterminal (Norge)
Tid: 12:15:00, Destination: Kungsbacka station
Tid: 12:16:00, Destination: Stenungsund station
Tid: 12:16:00, Destination: Göteborg Östra sjukhuset
Tid: 12:16:00, Destination: Kortedala Aprilgatan (Göteborg kn)
Tid: 12:17:00, Destination: Mölnlycke station (Härryda kn)
Tid: 12:17:00, Destination: Göteborg Heden
Tid: 12:18:00, Destination: Stockholm Centralstation
Tid: 12:18:00, Destination: Stenared (Göteborg kn)
Tid: 12:18:00, Destination: Amhult Resecentrum (Göteborg kn)
Tid: 12:19:00, Destination: Mölnlycke station (Härryda kn)